In [6]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans


# Read the CSV file
#d = pd.read_csv("MaternalHealthRiskDataSet.csv")
#d = pd.read_csv("wine.csv")
d = pd.read_csv("COAD.csv")

#target_c = 'class'
target_c = 'target'
dd = dd = d.drop([target_c], axis=1)
#dataset dependant variables
types = d[target_c].unique()
types

array([1, 0])

In [7]:
#split data into test & train to validade clusters

X_train, X_test, y_train, y_test = train_test_split(dd, d[[target_c]],
                                                    test_size=0.33, random_state=0)

ytrain_l = y_train[target_c].tolist()
ytest_l = y_test[target_c].tolist()

In [8]:
#normalize data

scaler = MinMaxScaler()
scaler.fit(X_train)

train_df = scaler.transform(X_train)
test_df = scaler.transform(X_test)

test_df

array([[0.42999382, 0.1891831 , 0.18911616, ..., 0.12093476, 0.07087319,
        0.15280787],
       [0.04438188, 0.0412836 , 0.08284641, ..., 0.13264012, 0.06092087,
        0.05468525],
       [0.04479388, 0.01751046, 0.07683109, ..., 0.06065431, 0.06684924,
        0.05747413],
       ...,
       [0.44638238, 0.88083369, 0.2167672 , ..., 0.27606159, 0.53343663,
        0.24321841],
       [0.04090274, 0.07044265, 0.41819536, ..., 0.38558773, 0.22430403,
        0.1823291 ],
       [0.02243128, 0.10215687, 0.26513228, ..., 0.39845614, 0.26776554,
        0.21519878]])

In [12]:
#create and train model
k = 3#number of clusters
kmeans = KMeans(n_clusters = k, random_state = 0, n_init=10)

kmeans.fit(train_df)

print(kmeans.cluster_centers_)
print(kmeans.labels_)

[[0.1276332  0.24179976 0.38570706 ... 0.47324727 0.42569184 0.34708729]
 [0.0614086  0.05957106 0.07124085 ... 0.12196062 0.09232143 0.06830795]
 [0.12568227 0.17785785 0.31256405 ... 0.33218051 0.3145279  0.21489937]]
[1 0 2 2 0 1 2 1 1 1 1 2 1 0 2 2 2 2 1 1 0 2 2 1 1 0 1 1 2 2 2 1 1 1 2 2 2
 1 2 1 2 2 1 2 2 0 1 1 0 0 0 0 2 1 2 2 2 1 0 1 2 2 0 2 0 1 2 2 2 1 2 2 1 1
 1 1 0 2 0 2 1 1 1 2 2 1 1 1 1 1 1 2 0 1 2 1 2 2 1 0 0 1 2 1 2 2 0 2 1 2 1
 2 2 2 2 2 0 1 2 2 2 1 2 2 1 1 2 1 0 0 1 2 1 1 2 2 1 2 2 2 1 2 2 1 2 1 1 2
 1 2 1 2 1 1 1 1 2 2 1 2 0 0 1 1 1 2 1 1 1 0 2 2 1 2 2 2 1 1 2 2 0 0 2 1 1
 2 2 1 1 1 0 1 1 1 2 1 2 2 1 1 1 2 1 2 2 1 2 0 2 0 2 1 1 1 2 2 1 1 1 1 1 0
 1 2 1 1 2 2 1 1 2 0 1 2 2 2 2 2 2 1 2 2 1 1 0 0 2 2 0 0 1 2 0 0 1 1 2 2 1
 2 1 2 2 0 1 1 0 0 1 0 2 1 1 2 2 2 2 1]


In [13]:

#initialize list to evaluate
res = []
r = []
rt = [0] * k
ra = [0] * k

for i in range(k):
    tmp = dict()
    for j in types:
        tmp[j] = list((0,0))
    res.append(tmp)

for i in range(len(ytrain_l)):
    #print(kmeans.labels_[i],ytrain_l[i])
    res[kmeans.labels_[i]][ytrain_l[i]][0] += 1

for i in range(k):
    o = str(i)
    m = ''
    mm = 0
    for j in types:
        o += "\t" + str(res[i][j][0])
        if res[i][j][0] > mm:
            mm = res[i][j][0]
            m = j
            #print(m,mm)
    o += "\t" + str(m)
    r.append(m)
    
    
#Test model
y_pred = kmeans.predict(test_df)

for i in range(len(y_pred)):
    #print(i,y_pred[i],ytest_l[i])
    rt[y_pred[i]] += 1
    res[y_pred[i]][ytest_l[i]][1] += 1
    if(ytest_l[i] == r[y_pred[i]]):
        ra[y_pred[i]] += 1
    
#print(res)

rf = 0
for i in range(k):
    o = 0
    if rt[i]:
        o = ra[i]/rt[i]
    rf += o
    #print(str(i) + "\t" + r[i] +  "\t" + str(o))
    
print("%1d\t%.3f" % (k,rf/k))

3	0.764


In [11]:
#choosing best k on test

#for k in range(5,100,5):#number of clusters
for k in range(2,10,1):#number of clusters
    kmeans = KMeans(n_clusters = k, random_state = 0, n_init=10)
    kmeans.fit(train_df)

    #initialize list to evaluate
    res = []
    r = []
    rt = [0] * k
    ra = [0] * k

    for i in range(k):
        tmp = dict()
        for j in types:
            tmp[j] = list((0,0))
        res.append(tmp)

    for i in range(len(ytrain_l)):
        #print(kmeans.labels_[i],ytrain_l[i])
        res[kmeans.labels_[i]][ytrain_l[i]][0] += 1

    for i in range(k):
        o = str(i)
        m = ''
        mm = 0
        for j in types:
            o += "\t" + str(res[i][j][0])
            if res[i][j][0] > mm:
                mm = res[i][j][0]
                m = j
                #print(m,mm)
        o += "\t" + str(m)
        r.append(m)


    #Test model
    y_pred = kmeans.predict(test_df)

    for i in range(len(y_pred)):
        #print(i,y_pred[i],ytest_l[i])
        rt[y_pred[i]] += 1
        res[y_pred[i]][ytest_l[i]][1] += 1
        if(ytest_l[i] == r[y_pred[i]]):
            ra[y_pred[i]] += 1

    #print(res)

    rf = 0
    for i in range(k):
        o = 0
        if rt[i]:
            o = ra[i]/rt[i]
        rf += o
        #print(str(i) + "\t" + r[i] +  "\t" + str(o))

    print("%1d\t%.3f" % (k,rf/k))

2	0.823
3	0.764
4	0.733
5	0.611
6	0.660
7	0.578
8	0.586
9	0.455
